# COPT and LOLP Calculator

## **Usage**

1. In Google Colab, press `ctrl+F9`
1. Press `run anyway` if a warning message appear.
1. Wait for the UI to show up (until a loading circle in the bottom of notebook stopped).
1. Upload files by pressing upload button.
1. Press compute to run simulation.
1. Press `ctrl+F7` to restart.

## **Input Format**

### Forced outage rate input format

|No |Unit Name  |Capacity (MW)  |FOR (Outage)   |Status |
|---|-----------|---------------|---------------|-------|
|1  |COAL-1     |50             |0.1            |1      |
|2  |COAL-2     |20             |0.1            |0      |
|3  |COAL-3     |50             |0.2            |1      |
|4  |COAL-4     |30             |0.1            |1      |

Note:

1. status denotes that the generating unit is considered (1) or not (0) in making COPT table.

### Profile input format

|No     |Demand (MW)    |
|-------|---------------|
|1      |150            |
|...    |...            |
|...    |...            |
|8760   |125            |

### VOLL input format

VOLL can use `_` and `,` separator for thousands and `.` for cents, eg: `15_000_000.00`


In [ ]:
#@title
import itertools

import ipywidgets
from ipywidgets import widgets

import numpy as np
import pandas as pd


# get ipywidgets version
IPYWIDGETS_VERSION = ipywidgets.__version__.split('.')[0]

# setting numpy print option decimal places
decimal_places = 3
np.set_printoptions(precision=decimal_places, suppress=True)
pd.set_option('display.float_format', '{:.3f}'.format)

# lolp
def get_lolp(capacity, cumulative_probability, demand):
    """
    format:
        capacity (descend)
        cumulative_probability(descend)
    """
    try:
        idx = np.where(capacity < demand)[0][0]
    except IndexError:
        idx = -1
    return cumulative_probability[idx]

# eens
def get_eens(capacity, individual_probability, demand):
    # TODO: standardize input in teh form of one year data (8760 data) by default
    """
    format:
        capacity
        individual_probability
    """
    return sum(individual_probability
               * (capacity < demand)
               * (demand - capacity))

# Uploader class for input
class Uploader:
    def __init__(self, name=None):
        if name is None:
            self._name = ' '
        else:
            self._name = f' {name} '

        # uploader
        self._file_upload = widgets.FileUpload(
            accept='.csv*',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
            multiple=False  # True to accept multiple files upload else False
            )

        self.box = widgets.VBox(children=(self._file_upload,
                                           widgets.Label(value = f'Upload{self._name}in .csv Format')))

    def get_dataframe(self):
        return get_dataframe_from_widget(self._file_upload)

# widgets
output = widgets.Output()

def _get_content_and_name_ipywidget_8(file_upload):
    return (
        file_upload.value[0]['content'],
        file_upload.value[0]['name'],
    )

def _get_content_and_name_ipywidget_7(file_upload):
    for key, value in file_upload.value.items():
        return (
            value['content'],
            value['metadata']['name']
        )
    
if IPYWIDGETS_VERSION == '8':
    get_content_and_name = _get_content_and_name_ipywidget_8
elif IPYWIDGETS_VERSION == '7':
    get_content_and_name = _get_content_and_name_ipywidget_7
else:
    print('Please install either ipywidgets 7 or 8, for example:')
    print('pip install ipywidgets==7.7.2')

def get_content_and_name(uploader):
    for key, value in uploader.value.items():
        return value['content'], value['metadata']['name']

def write_file_from_bytes_data(bytes_data, file_name='NAME.csv'):
    with open(file_name, 'wb') as f:
        f.write(bytes_data)

def get_dataframe_from_widget(file_upload):
    # get content and file name
    content, name = get_content_and_name(file_upload)

    # from bytes data to csv file
    write_file_from_bytes_data(content, file_name=name)

    # pandas read csv
    return pd.read_csv(name)

# initiate widgets
for_data_uploader = Uploader(name='FOR Data')
demand_profile_data_uploader = Uploader(name='Demand Profile Data')
voll_widget = widgets.Text(
    value='15_000_000.00',
    description='VOLL: ',
    disabled=False
)
currency_widget = widgets.Dropdown(
    options=['IDR', 'USD'],
    value='IDR',
    description='Currency: ',
    disabled=False,
)

# button action
def on_button_clicked(b):
    with output:
        # COPT
        df = for_data_uploader.get_dataframe()
        capacities = df['Capacity (MW)'].tolist()
        outage_rates = df['FOR (Outage)'].tolist()
        status = df['Status'].tolist()

        # used only for print input table
        # pd.DataFrame(data={'capacities': capacities,
        #                    'outage_rates': outage_rates,
        #                    'status': status},
        #              index=pd.RangeIndex(1, len(capacities) + 1, 1)).head(10)

        # filter only available generator
        generator_list = [[cap, 1-out]
                          for cap, out, stat
                          in sorted(zip(capacities, outage_rates, status), reverse=True)
                          if stat]

        # make tables for each generator
        # TODO: Tables already support derating, input data should also support it from excel
        tables = [np.array([generator, [0, 1 - generator[1]]]) for generator in generator_list]

        table = tables[0].copy()  # copy to avoid modifying data
        for table_ in tables[1:]:
            # faster than flatten + transpose
            table = np.hstack(((table[:, 0, None] + table_[:, 0]).reshape(-1,1),  # sum capacity
                               (table[:, 1, None] * table_[:, 1]).reshape(-1,1))  # multiply probability
                               )

            # sort table
            table = table[(-table[:, 0]).argsort(),:]

            # combine duplicate
            table = np.array([[k,sum([x[1] for x in list(g)])] 
                              for k,g in itertools.groupby(table, lambda x:x[0])])

            # TODO: Implement resample, useful for big COPT table, triggered only if max table length achieve

        table = np.hstack((table,
                           np.atleast_2d(np.cumsum(table[::-1,1])[::-1]).T,  # Cumulative Probability
                           np.atleast_2d(np.cumsum(table[:,1])).T))  # Reversed Cumulative Probability

        columns_name = ['Combined Capacity',
                        'Individual Probability',
                        'Cumulative Probability',
                        'Reversed Cumulative Probability']

        df = pd.DataFrame(data=table,
                          columns=columns_name,
                          index=pd.RangeIndex(1, len(table) + 1, 1, name='No'),
                          )

        # to save COPT in csv
        # CASE_NAME = 'COPT Case.csv'
        # df.to_csv(f'../results/COPT_{CASE_NAME}')

        df.head(10)  # to print copt table

        # LOLP
        demands = demand_profile_data_uploader.get_dataframe()['Demand (MW)'].values
        max_demand = np.max(demands)
        lolp = get_lolp(table[:,0], table[:,2], max_demand)

        print(f'LOLP (using anual peak load): {lolp * 100:.6f} %')

        # LOLE
        print(f'LOLE (using anual peak load): {lolp * 365:.6f} day/year')

        # EENS
        eens = [get_eens(table[:,0], table[:,1], demand) for demand in demands]

        print(f'Total EENS (using 8765 demand profile): {sum(eens):.2f} MW')

        # Expected Unserved Energy Cost
        VOLL = float(voll_widget.value.replace(',',''))
        print(f'Total Expected Unserved Energy Cost from EENS: {currency_widget.value} {VOLL * sum(eens):,.2f}')

# initiate button    
button = widgets.Button(description="Compute!")
button.on_click(on_button_clicked)

# interface
widgets.VBox((for_data_uploader.box,
              demand_profile_data_uploader.box,
              voll_widget,
              currency_widget,
              button,
              output,
             ))